In [1]:
import pandas as pd
import boto3
from settings import *
from awstk import *
import io
from botocore.exceptions import ClientError

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', 
                        header=None, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])
df['class'] = df['class'].str.split('-').str[-1]
df['class'].value_counts(dropna=False)

class
setosa        50
versicolor    50
virginica     50
Name: count, dtype: int64

In [3]:
buffer = io.BytesIO()
df.to_parquet(buffer)
buffer.seek(0)
if not bucket_exists(bucket_name):
    s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': aws_region})
response = s3_client.put_object(Body=buffer, Bucket=bucket_name, Key=iris_data_key)

body = s3_client.get_object(Bucket=bucket_name, Key=iris_data_key)['Body']
buffer = io.BytesIO(body.read())
df_test = pd.read_parquet(buffer)
assert df.equals(df_test)